In [9]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import plotly.express as px
from policyengine_core.charts import format_fig
import plotly.graph_objects as go


In [10]:
def define_reforms():
    reform = Reform.from_dict({
        "gov.contrib.ubi_center.basic_income.amount.person.flat": {
            "2025-01-01.2025-12-31": 1160,
            "2026-01-01.2026-12-31": 1605,
            "2027-01-01.2027-12-31": 1686
        }
    }, country_id="us")

    reform_taxable = Reform.from_dict({
        "gov.contrib.states.or.rebate.state_tax_exempt": {
            "2024-01-01.2100-12-31": True
        },
        "gov.contrib.ubi_center.basic_income.amount.person.flat": {
            "2025-01-01.2025-12-31": 1160,
            "2026-01-01.2026-12-31": 1605,
            "2027-01-01.2027-12-31": 1686
        },
        "gov.contrib.ubi_center.basic_income.taxable": {
            "2024-01-01.2100-12-31": True
        }
    }, country_id="us")

    reform_flat_tax = Reform.from_dict({
        "gov.contrib.ubi_center.basic_income.amount.person.flat": {
            "2025-01-01.2025-12-31": 1160,
            "2026-01-01.2026-12-31": 1605,
            "2027-01-01.2027-12-31": 1686
        },
        "gov.contrib.ubi_center.flat_tax.rate.agi": {
            "2025-01-01.2025-12-31": 0.0187,
            "2026-01-01.2026-12-31": 0.025,
            "2027-01-01.2028-12-31": 0.0254
        }
    }, country_id="us")

    reform_taxable_flat_tax = Reform.from_dict({
        "gov.contrib.states.or.rebate.state_tax_exempt": {
            "2024-01-01.2100-12-31": True
        },
        "gov.contrib.ubi_center.basic_income.amount.person.flat": {
            "2025-01-01.2025-12-31": 1160,
            "2026-01-01.2026-12-31": 1605,
            "2027-01-01.2027-12-31": 1686
        },
        "gov.contrib.ubi_center.basic_income.taxable": {
            "2024-01-01.2100-12-31": True
        },
        "gov.contrib.ubi_center.flat_tax.rate.agi": {
            "2025-01-01.2025-12-31": 0.0187,
            "2026-01-01.2026-12-31": 0.025,
            "2027-01-01.2028-12-31": 0.0254
        }
    }, country_id="us")

    return reform, reform_taxable, reform_flat_tax, reform_taxable_flat_tax


In [11]:
# reform as tuples

In [12]:
def initialize_microsimulations(reform, reform_taxable, reform_flat_tax, reform_taxable_flat_tax):
    baseline = Microsimulation(dataset="cps_2024")
    reformed = Microsimulation(reform=reform, dataset="cps_2024")
    reformed_taxable = Microsimulation(reform=reform_taxable, dataset="cps_2024")
    reformed_flat_tax = Microsimulation(reform=reform_flat_tax, dataset="cps_2024")
    reformed_taxable_flat_tax = Microsimulation(reform=reform_taxable_flat_tax, dataset="cps_2024")
    
    return baseline, reformed, reformed_taxable, reformed_flat_tax, reformed_taxable_flat_tax


In [13]:
def calculate_poverty_impact_by_age(baseline, reformed, year):
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)

    baseline_poverty = baseline.calc("in_poverty", map_to="person", period=year)
    reform_poverty = reformed.calc("in_poverty", map_to="person", period=year)

    results = {}
    age_groups = [
        (0, 18, "0-17"),
        (18, 65, "18-64"),
        (65, 200, "65+"),
        (0, 200, "Overall")
    ]

    for min_age, max_age, label in age_groups:
        if label == "Overall":
            mask = (state_codes == "OR")
        else:
            mask = (state_codes == "OR") & (age >= min_age) & (age < max_age)

        baseline_poverty_group = baseline_poverty[mask].mean()
        reform_poverty_group = reform_poverty[mask].mean()

        relative_poverty_reduction = (reform_poverty_group - baseline_poverty_group) / baseline_poverty_group
        results[label] = relative_poverty_reduction
        print(baseline_poverty[(state_codes == "OR") & (age >= 65)]).mean()

    return results

In [21]:
# Initialize the microsimulation
baseline = Microsimulation(dataset="cps_2024")

# Set the years
years = [2025, 2026, 2027]

# Initialize a dictionary to store results
results = {}

for year in years:
    # Calculate the required variables
    state_codes = baseline.calc("state_code", map_to="person", period=year)
    age = baseline.calc("age", map_to="person", period=year)
    poverty_status = baseline.calc("in_poverty", map_to="person", period=year)

    # Create a mask for people in Oregon who are 65 or older
    mask = (state_codes == "OR") & (age >= 65)

    # Calculate the poverty rate
    oregon_senior_poverty_rate = pd.Series(poverty_status)[mask].sum()

    # Store the result
    results[year] = oregon_senior_poverty_rate

# Print the results
for year, rate in results.items():
    print(f"Baseline poverty rate for people 65 and older in Oregon in {year}: {rate:.2%}")

Baseline poverty rate for people 65 and older in Oregon in 2025: 4400.00%
Baseline poverty rate for people 65 and older in Oregon in 2026: 4600.00%
Baseline poverty rate for people 65 and older in Oregon in 2027: 4600.00%


In [16]:
def calculate_results_for_year(baseline, reformed, reformed_taxable, reformed_flat_tax, reformed_taxable_flat_tax, year):
    results_untaxed = calculate_poverty_impact_by_age(baseline, reformed, year)
    results_taxed = calculate_poverty_impact_by_age(baseline, reformed_taxable, year)
    results_flat_tax = calculate_poverty_impact_by_age(baseline, reformed_flat_tax, year)
    results_taxable_flat_tax = calculate_poverty_impact_by_age(baseline, reformed_taxable_flat_tax, year)

    year_results = []
    for age_group in results_untaxed.keys():
        year_results.append({
            "year": year,
            "age_group": age_group,
            "relative_poverty_reduction": results_untaxed[age_group],
            "relative_poverty_reduction_taxable": results_taxed[age_group],
            "relative_poverty_reduction_flat_tax": results_flat_tax[age_group],
            "relative_poverty_reduction_taxable_flat_tax": results_taxable_flat_tax[age_group]
        })

    return pd.DataFrame(year_results)

In [17]:
if __name__ == "__main__":
    # Step 1: Define reforms
    reform, reform_taxable, reform_flat_tax, reform_taxable_flat_tax = define_reforms()
    print("Reforms defined.")

    # Step 2: Initialize microsimulations
    baseline, reformed, reformed_taxable, reformed_flat_tax, reformed_taxable_flat_tax = initialize_microsimulations(reform, reform_taxable, reform_flat_tax, reform_taxable_flat_tax)
    print("Microsimulations initialized.")

    # Step 3: Calculate results year by year
    all_results = pd.DataFrame()
    for year in range(2205, 2028):
        print(f"Calculating results for year {year}...")
        year_results = calculate_results_for_year(baseline, reformed, reformed_taxable, reformed_flat_tax, reformed_taxable_flat_tax, year)
        all_results = pd.concat([all_results, year_results], ignore_index=True)
        
        # Save intermediate results
        intermediate_csv_path = f"or_rebate_{year}.csv"
        year_results.to_csv(intermediate_csv_path, index=False)
        print(f"Results for year {year} saved to {intermediate_csv_path}")

    # Step 4: Save final results
    final_csv_path = "or_rebate_all_years.csv"
    all_results.to_csv(final_csv_path, index=False)
    print(f"All results saved to {final_csv_path}")

    print("Calculation complete. You can now proceed with creating plots or further analysis.")

Reforms defined.


KeyboardInterrupt: 